In [1]:
from gensim import corpora, models
from normalization import normalize_corpus
import numpy as np

paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [2]:
toy_corpus = ["The fox jumps over the dog",
"The fox is very clever and quick",
"The dog is slow and lazy",
"The cat is smarter than the fox and the dog",
"Python is an excellent programming language",
"Java and Ruby are other programming languages",
"Python and Java are very popular programming languages",
"Python programs are smaller than Java programs"]

# LATENT SEMANTIC INDEXING

In [4]:
# LSI topic model
norm_tokenized_corpus = normalize_corpus(toy_corpus, tokenize=True)
norm_tokenized_corpus

dictionary = corpora.Dictionary(norm_tokenized_corpus)
print (dictionary.token2id)

corpus = [dictionary.doc2bow(text) for text in norm_tokenized_corpus]
corpus

tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

total_topics = 2

lsi = models.LsiModel(corpus_tfidf, 
                      id2word=dictionary, 
                      num_topics=total_topics)
                      
for index, topic in lsi.print_topics(total_topics):
    print ('Topic #'+str(index+1))
    print (topic)
    print ()             
    


{'dog': 0, 'fox': 1, 'jump': 2, 'clever': 3, 'quick': 4, 'lazy': 5, 'slow': 6, 'cat': 7, 'smarter': 8, 'excellent': 9, 'language': 10, 'programming': 11, 'python': 12, 'java': 13, 'ruby': 14, 'popular': 15, 'program': 16, 'small': 17}
Topic #1
0.459*"programming" + 0.459*"language" + 0.344*"java" + 0.344*"python" + 0.336*"popular" + 0.318*"excellent" + 0.318*"ruby" + 0.148*"program" + 0.074*"small" + 0.000*"dog"

Topic #2
-0.459*"dog" + -0.459*"fox" + -0.444*"jump" + -0.322*"smarter" + -0.322*"cat" + -0.208*"slow" + -0.208*"lazy" + -0.208*"clever" + -0.208*"quick" + 0.000*"excellent"



In [6]:

def print_topics_gensim(topic_model, total_topics=1,
                        weight_threshold=0.0001,
                        display_weights=False,
                        num_terms=None):
    
    for index in range(total_topics):
        topic = topic_model.show_topic(index)
        topic = [(word, round(wt,2)) 
                 for word, wt in topic 
                 if abs(wt) >= weight_threshold]
        if display_weights:
            print ('Topic #'+str(index+1)+' with weights')
            print (topic[:num_terms] if num_terms else topic)
        else:
            print ('Topic #'+str(index+1)+' without weights')
            tw = [term for term, wt in topic]
            print (tw[:num_terms] if num_terms else tw)
        print ()
    


In [8]:
# print topics without weights
print_topics_gensim(topic_model=lsi,
                    total_topics=total_topics,
                    num_terms=5,
                    display_weights=True)

Topic #1 with weights
[('programming', 0.46), ('language', 0.46), ('java', 0.34), ('python', 0.34), ('popular', 0.34)]

Topic #2 with weights
[('dog', -0.46), ('fox', -0.46), ('jump', -0.44), ('smarter', -0.32), ('cat', -0.32)]

